# 1. Imports and inputs

In [ ]:
import os
import sys
import shutil

# insert the directory where you extracted the zip file here:
git_dir = r"C:\Work\Projects\HL-P24050_ROI\05_Analysis\P1414"

sys.path.append(os.path.join(git_dir, 'Code'))
sys.path.append(os.path.join(git_dir, 'HydroLogic_Inundation_toolbox'))
sys.path.append(os.path.join(git_dir, 'HydroLogic_Inundation_toolbox/Readers'))

from notebooks.LDO_export_scripts.ldo_metadata import MetadataGenerator

#================== Inputs ==================#

model_dir = r"C:\Work\Projects\HL-P24050_ROI\05_Analysis\SAS_workdir\run_model\test_dambreach\nothing"

file_locations = {
    'fm_dir': "dflowfm",
    'output_dirname': "output",
    'mdu_file': "DFM.mdu",
    'map_file': "map.nc",
    'his_file': "DFM_his.nc",
    'structures_file': "structures.ini",
    'dia_file': "DFM.dia",
}

#============================================#

# 2. Generate metadata

In [5]:

metadatagenerator = MetadataGenerator(model_dir, file_locations)
df = metadatagenerator.execute()
metadatagenerator.write(write_as_excel=True)

clean_df_for_notebook =  df.loc[:, ~df.iloc[-1].isna()]
clean_df_for_notebook

2024-10-22 15:29:43,317 - INFO - Succesfully wrote the LDO metadata to: C:\Work\Projects\HL-P24050_ROI\05_Analysis\SAS_workdir\run_model\test_dambreach\nothing\dflowfm\output\LDO\LDO_metadata.csv


,"(Scenario, Scenario Identificatie)","(Scenario, Scenariodatum)","(Location, x-coordinaten doorbraaklocatie)","(Location, y-coordinaten doorbraaklocatie)","(Bres, Materiaal kering)","(Bres, Bresdiepte)","(Bres, Duur bresgroei in verticale richting)","(Bres, Initiele bresbreedte)","(Bres, Startmoment bresgroei)","(Bres, Maximale bresbreedte)",...,"(Bres, Maximaal bresdebiet)","(Bres, Maximale instroom)","(Buitenwater, Maximale buitenwaterstand)","(Model, Start berekening)","(Model, Einde berekening)","(Model, Rekenduur)","(Model, Start simulatie)","(Model, Einde simulatie)","(Model, Duur)","(Model, Modelversie)"
0,Scenario,Scenario,Location,Location,Bres,Bres,Bres,Bres,Bres,Bres,...,Bres,Bres,Buitenwater,Model,Model,Model,Model,Model,Model,Model
1,Scenario Identificatie,Scenariodatum,x-coordinaten doorbraaklocatie,y-coordinaten doorbraaklocatie,Materiaal kering,Bresdiepte,Duur bresgroei in verticale richting,Initiele bresbreedte,Startmoment bresgroei,Maximale bresbreedte,...,Maximaal bresdebiet,Maximale instroom,Maximale buitenwaterstand,Start berekening,Einde berekening,Rekenduur,Start simulatie,Einde simulatie,Duur,Modelversie
2,nothing,2024-10-22,110264.6021,435441.3475,zand,4.5,0 d 00:06,5.0,0 d 00:00,59.0,...,1802.844044,7023317.173818,4.0,18/10/2024 14:57,18/10/2024 15:01,0 d 00:03,01/01/2000 00:00,01/01/2000 04:00,0 d 04:00,1.2.167.141798


# 3. Generate TIF files